# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,las lomas,8.4333,8.4333,80.01,78.39,100,13.00,PA,1694129317
1,1,invercargill,-46.4000,-46.4000,56.53,56.53,47,5.88,NZ,1694129197
2,2,eureka,40.8021,40.8021,80.74,63.21,31,5.01,US,1694129318
3,3,balabac,7.9868,7.9868,79.56,79.56,100,8.46,PH,1694129318
4,4,strazhitsa,43.2333,43.2333,57.00,57.00,92,2.48,BG,1694129319


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM")

                     
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values

filtered_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<90)
                                   & (city_data_df["Max Temp"]>70)
                                   & (city_data_df["Cloudiness"]<50)
                                   & (city_data_df["Wind Speed"]>10.0)
                                   & (city_data_df["Humidity"]<70)]

# Drop any rows with null values
filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,praia da vitoria,38.7333,38.7333,70.05,68.76,20,15.17,PT,1694129362
165,165,lompoc,34.6391,34.6391,76.78,67.59,20,10.36,US,1694129367
258,258,salinas,36.6777,36.6777,82.99,61.50,0,13.80,US,1694129233
494,494,fortuna,40.5982,40.5982,71.92,62.94,20,14.97,US,1694129587
539,539,santa barbara,34.7333,34.7333,81.16,64.42,0,12.66,US,1694129610


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=filtered_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
84,praia da vitoria,PT,38.7333,38.7333,68.76,
165,lompoc,US,34.6391,34.6391,67.59,
258,salinas,US,36.6777,36.6777,61.50,
494,fortuna,US,40.5982,40.5982,62.94,
539,santa barbara,US,34.7333,34.7333,64.42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
              "apiKey":geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
praia da vitoria - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
salinas - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
santa barbara - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
84,praia da vitoria,PT,38.7333,38.7333,68.76,No hotel found
165,lompoc,US,34.6391,34.6391,67.59,No hotel found
258,salinas,US,36.6777,36.6777,61.50,No hotel found
494,fortuna,US,40.5982,40.5982,62.94,No hotel found
539,santa barbara,US,34.7333,34.7333,64.42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points("Lng",
                                        "Lat",
                                     geo=True,
                                        color="City",
                                      alpha=1,
                            size = "Humidity",
                                tiles = "OSM",
          hover_cols=["Hotel Name","Country"],
                            frame_width = 700,
                           frame_height = 700)
                                     

# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)